# 6.2 Tiktoken

기본적으로, 이전에 배웠던 모든 splitter들은 텍스트의 length를 계산해서 한 덩어리의(chunk) 크기를 알아낸다. 그 작업에 파이썬 표준 라이브러리가 지원하는 표준 len함수를 사용한다. 따라서 다음과 같이 splitter클래스를 만들때 나만의 len함수를 명시적으로 설정할 수 있다. 
```python
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=50,
    length_function=len,
)
```
이렇게 설정하면 우리의 텍스트가 얼마나 많은 문자를 갖고 있는지 세주게 된다. 예를들어, 'ABCD'를 입력하면, len이 문자의 개수인 '4'를 반환할 것이다.

하지만 사실 LLM들은 Token을 이런 방법으로 세지 않는다. token은 문자(letter)와 같은 의미가 아니다. 가끔은 문자 두개, 때로는 세 개가 한개의 token으로 취급된다. 이는 OpenAI 웹사이트에서 예시를 보면 더 이해가 쉽다. 여기에선 model의 관점에서 우리가 몇개의 token을 사용하는지 볼 수 있다. 즉, token과 문자들(characters)의 차이를 알 수 있다.

https://platform.openai.com/tokenizer

위 사이트에 들어가서 내가 알고 싶은 문장을 치면 토큰과 문자의 차이를 바로 알 수 있다. 예를들어 'Hello my name is Kangsik"라는 문자를 치면 토큰이 몇개인지, 문자가 몇개인지 어떻게 구분해서 개수를 세었는지 등이 나온다.

그리고 TOKEN IDS를 볼 수 있는데, 이것이 바로 model이 실제로 보고 있는것이다. 우리가 입력해준 텍스트들은, model에게는 token ids로 보인다.

웹사이트에 나와있는 설명을 읽어보면 model은 token간의 통계적인 관계를 이해하도록 학습을 한다고 한다.



우리는 이 tokenizer함수를 사용해서 splitter에 적용해 보고 싶다. 다음과 같이 하면 된다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

여기서 tiktoken_encoder는 openAI에 의해 만들어진 것이다. 이렇게 from_tiktoken_encoder라는 클래스 메서드를 사용하면 모델이 텍스트를 세는 방법과 우리가 텍스트를 세는 방법이 일치하게 된다.

model에는 한계(limit)가 있어서, 원하는 텍스트들을 모두 한번에 입력할 수 없다. 그래서 우리 텍슽트의 길이를 계산할 때 model과 같은 방법으로 계산하는게 좋다.